In [ ]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [ ]:
reload(pi)
reload(pi.strategies.buy_x_week_low)
reload(pi.summarize.trades)
reload(pi.indicators)

In [ ]:
symbols = ['A','AA']

In [ ]:
#df = gas.get_symbols_df(symbols)

In [ ]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col(pi.indicators.DATE_COLUMN) >= datetime(2020, 1, 1))
        df = df.collect()

In [ ]:
saved_df = df.clone()

In [ ]:
sma100 = pi.indicators.simple_moving_average(df, 100)
df = sma100.df
slope_up = pi.indicators.slope_up(df, column=sma100.column)
df = slope_up.df.with_columns(pl.col(slope_up.column).shift()) #shifting so we use previous bar

In [ ]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))
df

In [ ]:
#filter out average price too low
df = df.filter(pl.col(pi.indicators.CLOSE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 3)

# filter out too many 0 volume bars. This gets rid of a lot of garbage
df = df.filter((pl.col(pi.indicators.VOLUMNE_COLUMN) == 0).sum().over(pi.indicators.SYMBOL_COLUMN) < 3)

# minimum amount of volume
df = df.filter(pl.col(pi.indicators.VOLUMNE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 100000)


In [ ]:
lookback_bars = 250
offset_bars = 10
modresult_05 = pi.strategies.buy_x_week_low.strategy(df, lookback_bars, offset_bars, 0.5, 5, 2, 3)
modresult_1 = pi.strategies.buy_x_week_low.strategy(df, lookback_bars, offset_bars, 1, 5, 2, 3)

In [ ]:
trades_05 = pi.summarize.trades.Trades.from_strategy_result(modresult_05, [slope_up.column])
trades_1 = pi.summarize.trades.Trades.from_strategy_result(modresult_1, [slope_up.column])

In [ ]:
trades_05.summarize_strategy()

In [ ]:
trades_1.summarize_strategy()

In [ ]:
trades_1.df.filter(pl.col("Gain/Loss%") < -70.54)

In [ ]:
trades_05.df.filter(pl.col("Highest%") > 900.73)

In [ ]:
tf = trades_05.df.groupby(pi.indicators.SYMBOL_COLUMN).agg(pl.col(pi.indicators.SYMBOL_COLUMN).count().alias("trades")).sort("trades", descending=True)
tf

In [ ]:
trades_05.df

In [ ]:
index = "index"
t = df.with_row_count(index).filter(pl.col(index) < 10).select(pl.col(pi.indicators.DATE_COLUMN),pi.indicators.LOW_COLUMN)

In [ ]:
t.with_columns(pl.col("Low").rolling_min(2).shift(3).alias("roll"))